<a href="https://colab.research.google.com/github/Kanyi254/Insurance-quotation/blob/Brandon/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd

In [25]:
# Load the full datasets for Q3 and Q4
file_path_q4 = 'BICOR LIFE - Medical Quota share - 4th qtr 2020.xlsx'
file_path_q3 = 'Medical quota share treaty  3rd Qtr 2020.xlsx'

xls_q4 = pd.ExcelFile(file_path_q4)
xls_q3 = pd.ExcelFile(file_path_q3)

premium_q4_full = pd.read_excel(xls_q4, sheet_name='Premium Bordereaux')
claims_q4_full = pd.read_excel(xls_q4, sheet_name='Claims Bordereaux')
premium_q3_full = pd.read_excel(xls_q3, sheet_name='Premium Bordereaux')
claims_q3_full = pd.read_excel(xls_q3, sheet_name='Claims Bordereaux')

# Display column names to inspect
print("Premium Q4 Columns:", premium_q4_full.columns)
print("Claims Q4 Columns:", claims_q4_full.columns)
print("Premium Q3 Columns:", premium_q3_full.columns)
print("Claims Q3 Columns:", claims_q3_full.columns)

Premium Q4 Columns: Index(['Policy Holder ID', 'Principal beneficiary', 'Dependants',
       'Total beneficiaries', 'Police ID', 'Start Date of Cover',
       'End Date of Cover', ' ', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Premium Amount', 'Benefit Limit', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Premium Paid/Billed'],
      dtype='object')
Claims Q4 Columns: Index(['Policy Holder ID', 'Member ID', 'Start Date of Cover',
       'End Date of Cover', 'Unnamed: 4', 'Unnamed: 5',
       'Date of Claim/Treatment Date', 'Date of Payment/Approval Date',
       'Amount Claimed', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Amount Paid', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Benefit Limit', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Provider Name/Code'],
      dtype='object')
P

In [41]:
# Get information about each sheet in the Excel files
print("Q4 Excel File Sheets:")
for sheet_name in xls_q4.sheet_names:
  try:
    df = pd.read_excel(xls_q4, sheet_name=sheet_name)
    print(f"- {sheet_name}:")
    print(f"  - Shape: {df.shape}")
    print(f"  - Columns: {df.columns.tolist()}")
    print(f"  - Data types: {df.dtypes}")
    print(f"  - Sample data:")
    print(df.head())
    print("-" * 20)
  except Exception as e:
    print(f"Error reading sheet {sheet_name}: {e}")

print("\nQ3 Excel File Sheets:")
for sheet_name in xls_q3.sheet_names:
  try:
    df = pd.read_excel(xls_q3, sheet_name=sheet_name)
    print(f"- {sheet_name}:")
    print(f"  - Shape: {df.shape}")
    print(f"  - Columns: {df.columns.tolist()}")
    print(f"  - Data types: {df.dtypes}")
    print(f"  - Sample data:")
    print(df.head())
    print("-" * 20)
  except Exception as e:
    print(f"Error reading sheet {sheet_name}: {e}")

Q4 Excel File Sheets:
- Premium Bordereaux:
  - Shape: (19, 20)
  - Columns: ['Policy Holder ID', 'Principal beneficiary', 'Dependants', 'Total beneficiaries', 'Police ID', 'Start Date of Cover', 'End Date of Cover', ' ', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Premium Amount', 'Benefit Limit', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Premium Paid/Billed']
  - Data types: Policy Holder ID                 object
Principal beneficiary           float64
Dependants                       object
Total beneficiaries             float64
Police ID                       float64
Start Date of Cover              object
End Date of Cover        datetime64[ns]
                                 object
Unnamed: 8                       object
Unnamed: 9                       object
Unnamed: 10                      object
Unnamed: 11                      object
Premium Amount                  float64
Benefit Limit                    object
Unnamed: 14

In [ ]:
# prompt: what can i do to the NaN s

# Assuming you have your DataFrame loaded as 'df'

# 1. Identify NaN values:
nan_counts = df.isnull().sum()
print("NaN counts per column:\n", nan_counts)

# 2. Decide how to handle NaNs based on the context:

# a) Drop rows/columns with NaNs:
# df_dropped = df.dropna()  # Drop rows with any NaN
# df_dropped = df.dropna(axis=1)  # Drop columns with any NaN

# b) Fill NaNs with a specific value (e.g., mean, median, 0):
# df_filled = df.fillna(0) # Replace NaN with 0
# mean_value = df['column_name'].mean()
# df_filled = df.fillna({'column_name': mean_value}) # Replace NaN in column with its mean

# c) Forward or backward fill NaNs (useful for time series data):
# df_filled = df.fillna(method='ffill')  # Forward fill
# df_filled = df.fillna(method='bfill')  # Backward fill

# d) Interpolate missing values:
# df_interpolated = df.interpolate()

# Example: Filling NaN values in a specific column with the mean:
# if 'column_with_nan' in df.columns:
#   mean_value = df['column_with_nan'].mean()
#   df['column_with_nan'] = df['column_with_nan'].fillna(mean_value)

# Choose the method that is most appropriate for your dataset and the analysis you want to perform.

In [26]:
# Drop columns that are mostly empty or irrelevant
premium_q4_clean = premium_q4_full.dropna(axis=1, how='all')
claims_q4_clean = claims_q4_full.dropna(axis=1, how='all')
premium_q3_clean = premium_q3_full.dropna(axis=1, how='all')
claims_q3_clean = claims_q3_full.dropna(axis=1, how='all')

In [27]:
# Convert 'Policy Holder ID' to string type
claims_q4_clean['Policy Holder ID'] = claims_q4_clean['Policy Holder ID'].astype(str)

# Alternatively for Q3 dataset:
premium_q4_clean['Policy Holder ID'] = premium_q4_clean['Policy Holder ID'].astype(str)


<ipython-input-27-195a1889571e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims_q4_clean['Policy Holder ID'] = claims_q4_clean['Policy Holder ID'].astype(str)


In [28]:
# Handle missing values by dropping rows with missing premium or claims data
# Replace with actual column names from your dataset
premium_q4_clean = premium_q4_clean.dropna(subset=['Premium Amount', 'Premium Paid/Billed']) # Replaced 'Premium', 'Paid to Date' with actual column names 'Premium Amount', 'Premium Paid/Billed'

# Changed 'Incurred to date' to 'Amount Claimed' based on column names provided
claims_q4_clean = claims_q4_clean.dropna(subset=['Amount Claimed', 'Policy Holder ID'])

premium_q3_clean = premium_q3_clean.dropna(subset=['Premium Amount', 'Premium Paid/Billed']) # Replaced 'Premium', 'Paid to Date' with actual column names 'Premium Amount', 'Premium Paid/Billed'
# Assuming 'Amount Claimed' is the equivalent for Q3 as well - adjust if needed
claims_q3_clean = claims_q3_clean.dropna(subset=['Amount Claimed', 'Policy Holder ID'])

# Display the cleaned data
print(premium_q4_clean.head())
print(claims_q4_clean.head())
print(premium_q3_clean.head())
print(claims_q3_clean.head())


               Policy Holder ID  Principal beneficiary Dependants  \
1           NDUWUMUKAMA CHARLES                    1.0          4   
2      NSABIYUMVA JEAN BERTRAND                    1.0          2   
3   BICOR VIE ET CAPITALISATION                   20.0         42   
4      NDABARUSHIMANA DIEUDONNE                    1.0          4   
14       BURUNDI LEND AND LEASE                    1.0          1   

    Total beneficiaries   Police ID  Start Date of Cover End Date of Cover  \
1                   5.0  20000030.0  2020-10-22 00:00:00        2021-10-21   
2                   3.0  19000002.0  2020-10-23 00:00:00        2021-10-22   
3                  62.0  19000005.0  2020-11-05 00:00:00        2021-11-04   
4                   5.0  20000031.0  2020-12-04 00:00:00        2021-12-03   
14                  2.0  20000012.0        November 2020        2021-05-04   

             Unnamed: 8 Unnamed: 9 Unnamed: 10 Unnamed: 11  Premium Amount  \
1     948885          1     948885    

In [32]:
# Remove rows with non-numeric Policy Holder IDs
claims_q4_clean = claims_q4_clean[pd.to_numeric(claims_q4_clean['Policy Holder ID'], errors='coerce').notnull()]

# Convert to numeric (if needed)
claims_q4_clean['Policy Holder ID'] = pd.to_numeric(claims_q4_clean['Policy Holder ID'])


In [45]:
# Updated query function to handle non-string values in 'Policy Holder ID'
def query_claims_by_group(data, policy_holder_group=None, min_claim=None, max_claim=None, provider=None):
    """
    Function to query claims for a specific group (Policy Holder ID like Alliance) and other parameters.
    :param data: DataFrame containing the claims data
    :param policy_holder_group: Filter by Policy Holder ID (group like 'Alliance')
    :param min_claim: Minimum claim amount
    :param max_claim: Maximum claim amount
    :param provider: Filter by provider name or code
    :return: Queried DataFrame
    """
    # Ensure 'Policy Holder ID' column is string type to prevent .str accessor issues
    data['Policy Holder ID'] = data['Policy Holder ID'].astype(str)

    # Filter by Policy Holder Group (e.g., 'Alliance')
    if policy_holder_group:
        data = data[data['Policy Holder ID'].str.contains(policy_holder_group, case=False, na=False)]

    # Filter by minimum and maximum claim amounts
    if min_claim:
        data = data[data['Amount Paid'] >= min_claim]
    if max_claim:
        data = data[data['Amount Paid'] <= max_claim]

    # Filter by provider name/code
    if provider:
        data = data[data['Provider Name/Code'].str.contains(provider, case=False, na=False)]

    return data

# Example usage: Query claims for policy holder group "ALLIANCE BURUNDAISE CONTRE LE SIDA" and claims above 5000
queried_claims = query_claims_by_group(claims_q4_clean, policy_holder_group='ALLIANCE BURUNDAISE CONTRE LE SIDA', min_claim=1000)
print(queried_claims)


Empty DataFrame
Columns: [Policy Holder ID, Member ID, Start Date of Cover, End Date of Cover, Date of Claim/Treatment Date, Date of Payment/Approval Date, Amount Claimed, Unnamed: 9, Unnamed: 10, Unnamed: 11, Unnamed: 12, Unnamed: 13, Amount Paid, Unnamed: 15, Unnamed: 16, Unnamed: 17, Unnamed: 18, Unnamed: 19, Unnamed: 20, Benefit Limit, Unnamed: 22, Unnamed: 23, Unnamed: 24, Unnamed: 25, Unnamed: 26, Provider Name/Code]
Index: []

[0 rows x 26 columns]
